In [ ]:
import math
import requests

def get_current_location():
    """
    Retrieves the current latitude and longitude based on the IP address.
    
    Returns:
        tuple: A tuple containing the latitude and longitude as floats.
    """
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        location = data["loc"].split(",")
        return float(location[0]), float(location[1])
    except Exception as e:
        print(f"Could not get the current location. Defaulting to 0, 0. Error: {e}")
        return 0.0, 0.0

def get_user_provided_location():
    """
    Prompts the user to input the latitude and longitude manually.

    Returns:
        tuple: A tuple containing the latitude and longitude as floats.
    """
    try:
        lat = float(input("Enter the latitude: "))
        lon = float(input("Enter the longitude: "))
        return lat, lon
    except ValueError as e:
        print(f"Invalid input. Defaulting to 0, 0. Error: {e}")
        return 0.0, 0.0

def generate_mavlink_mission(commands, current_lat, current_lon):
    """
    Generates a MAVLink mission based on the provided commands and starting location.

    Args:
        commands (list): A list of commands dicts with actions and parameters.
        current_lat (float): The starting latitude.
        current_lon (float): The starting longitude.

    Returns:
        str: A string representing the MAVLink mission.
    """
    mission_set = "QGC WPL 120\n"

    # Initialize waypoint index
    wp_index = 0

    # Add home position
    home_position = f"{wp_index}\t1\t0\t16\t0.000000\t0.000000\t0.000000\t0.000000\t{current_lat:.6f}\t{current_lon:.6f}\t0.000000\t1\n"
    mission_set += home_position
    wp_index += 1

    # Direction to azimuth mapping
    direction_to_azimuth = {
        "front": 0,
        "front-right": 45,
        "right": 90,
        "back-right": 135,
        "back": 180,
        "back-left": 225,
        "left": 270,
        "front-left": 315
    }

    # Loop through each command
    for command in commands:
        if command["action"].lower() == "takeoff":
            # Takeoff command
            mission_set += f"{wp_index}\t0\t3\t22\t0.000000\t0.000000\t0.000000\t0.000000\t{current_lat:.6f}\t{current_lon:.6f}\t{command['altitude']:.6f}\t1\n"
            wp_index += 1
        elif command["action"].lower() == "move":
            distance = command['distance']
            direction = command['direction']

            if direction in direction_to_azimuth:
                azimuth = direction_to_azimuth[direction]
                # Calculate the new position based on distance and azimuth
                earth_radius = 6371000  # Radius of Earth in meters

                # Convert distance to radians
                distance_rad = distance / earth_radius

                # Convert azimuth to radians
                azimuth_rad = math.radians(azimuth)

                # Calculate new latitude
                new_lat = math.asin(math.sin(math.radians(current_lat)) * math.cos(distance_rad) +
                                    math.cos(math.radians(current_lat)) * math.sin(distance_rad) * math.cos(azimuth_rad))
                new_lat = math.degrees(new_lat)

                # Calculate new longitude
                new_lon = math.radians(current_lon) + math.atan2(math.sin(azimuth_rad) * math.sin(distance_rad) * math.cos(math.radians(current_lat)),
                                                                 math.cos(distance_rad) - math.sin(math.radians(current_lat)) * math.sin(math.radians(new_lat)))
                new_lon = math.degrees(new_lon)

                # Move command with new position
                mission_set += f"{wp_index}\t0\t3\t16\t0.000000\t0.000000\t0.000000\t0.000000\t{new_lat:.6f}\t{new_lon:.6f}\t0.000000\t1\n"
                wp_index += 1

                # Update current position
                current_lat, current_lon = new_lat, new_lon
            else:
                print(f"Invalid direction: {direction}")
        elif command["action"].lower() == "land":
            # Land command
            mission_set += f"{wp_index}\t0\t3\t21\t0.000000\t0.000000\t0.000000\t0.000000\t{current_lat:.6f}\t{current_lon:.6f}\t0.000000\t1\n"
            wp_index += 1
        else:
            print(f"Invalid action: {command['action']}")

    return mission_set

def save_mavlink_mission(mission_set):
    """
    Saves the MAVLink mission to a text file.

    Args:
        mission_set (str): The MAVLink mission string.
    """
    if mission_set:
        filename = "Mission.txt"
        with open(filename, "w") as file:
            file.write(mission_set)
        print(f"MAVLink mission set saved to '{filename}'")

def chat_with_follow_up():
    """
    Main function to interact with the user, get commands, and generate/save the mission.
    """
    # Ask the user if they want to use auto-generated coordinates or input their own
    use_auto_coordinates = input("Do you want to use auto-generated coordinates based on your current location? (yes/no): ").strip().lower()

    if use_auto_coordinates == 'yes':
        # Get the current location of the computer
        current_lat, current_lon = get_current_location()
    else:
        # Get the user-provided location
        current_lat, current_lon = get_user_provided_location()

    print(f"Starting Latitude: {current_lat}, Starting Longitude: {current_lon}")

    print("\nNote: The movement directions are based on the drone being oriented north.")

    # Ask the user for commands
    commands = []
    while True:
        action = input("What action do you want the drone to perform? (takeoff/move/land/stop): ")
        if action.lower() == "stop":
            break
        elif action.lower() in ["takeoff", "move", "land"]:
            if action.lower() == "takeoff":
                altitude = input("Enter the altitude for takeoff (in meters): ")
                commands.append({"action": action, "altitude": float(altitude)})
            elif action.lower() == "move":
                distance = input("Enter the distance to move (in meters): ")
                direction = input("Enter the direction to move (front/front-right/right/back-right/back/back-left/left/front-left): ").lower()
                commands.append({"action": action, "distance": float(distance), "direction": direction})
            else:
                commands.append({"action": action})
        else:
            print("Invalid action. Please enter 'takeoff', 'move', 'land', or 'stop'.")

    # Generate MAVLink mission set
    mission_set = generate_mavlink_mission(commands, current_lat, current_lon)

    # Save MAVLink mission set to a text file
    save_mavlink_mission(mission_set)

# Example usage
chat_with_follow_up()
